In [1]:
import os
import pandas as pd
import numpy as np
from librosa import load as lr_load
import librosa as lr
from librosa import amplitude_to_db
from librosa.feature import melspectrogram, mfcc, chroma_stft, spectral_flatness, zero_crossing_rate, tempogram
from librosa.beat import tempo, beat_track
from librosa.feature import spectral_contrast as lf_spectral_contrast
from librosa.feature import spectral_rolloff as lf_spectral_rolloff
from librosa import pyin
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
import IPython.display as ipd

In [2]:
import os
import numpy as np
import pandas as pd
import librosa as lr
from IPython.display import display
from librosa import amplitude_to_db
from librosa.feature import chroma_stft, melspectrogram, mfcc, tempogram, zero_crossing_rate
from pydub import AudioSegment
from scipy.io import wavfile

C:\Users\molie\miniconda3\Lib\site-packages\pydub\utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


# Mel-Frequency Cepstral Coefficients (MFCCs) Calculation

1. **Framing:**
   $ x_w[n] = x[n] \cdot w[n] $

2. **Fast Fourier Transform (FFT):**
   $ X(k) = \text{FFT}(x_w[n]) $

3. **Mel Filterbank:**
   $ S_m = \sum_{k=0}^{N-1} |X(k)|^2 \cdot H_m(k) $

4. **Logarithmic Scaling:**
   $ M_m = \log(S_m) $

5. **Discrete Cosine Transform (DCT):**
   $ C_l = \sum_{m=0}^{M-1} \cos\left(\frac{\pi l (2m + 1)}{2M}\right) \cdot M_m $

   Symbols:
   - $ x[n] $: Audio signal.
   - $ w[n] $: Hamming window.
   - $ X(k) $: Result of the FFT.
   - $ H_m(k) $: $m$-th Mel filter.
   - $ S_m $: Energy in the $m$-th Mel filter.
   - $ M_m $: Log-Mel energy.
   - $ C_l $: $l$-th MFCC coefficient.
   - $ N $: Number of frequency points after FFT.
   - $ M $: Number of Mel filters.
   - $ L $: Number of desired MFCCs.

# 1. Fast Fourier Transform (FFT):
Explanation:
The Fast Fourier Transform (FFT) is an efficient algorithm for computing the Discrete Fourier Transform (DFT). It's like a speedy way to analyze the frequency content of a signal. Instead of taking a long time to calculate the DFT manually, the FFT quickly breaks down a sequence of data points into its frequency components.

Example:
Imagine you have a digital signal representing the vibrations of a guitar string over time. Applying the FFT allows you to rapidly identify the specific musical frequencies present in the sound. If the guitar string produces a clear note at a particular frequency, the FFT would efficiently highlight that frequency, making it easier to recognize the musical pitch and analyze the harmonic structure of the guitar sound.

# 2. Mel Filterbanks:
Explanation:
Mel Filterbanks are filters used in audio processing and speech recognition to mimic the human ear.

Example:
Consider an audio clip of someone playing a guitar. Using Mel Filterbanks, you can capture the essential features of the guitar sound that our ears find important. These filters might emphasize the frequency bands corresponding to the unique tones produced by the guitar strings.

# 3. Inverse Discrete Cosine Transform (IDCT):
Explanation:
The Inverse Discrete Cosine Transform (IDCT) is used to convert frequency domain information back to the time domain. It is often applied in image compression, where the image is transformed into the frequency domain using the Discrete Cosine Transform (DCT) for compression and then reconstructed back using the IDCT. It is often used with FFTs.

Example:
Think of a digital image of a sunset. The image can be compressed using the Discrete Cosine Transform (DCT). The IDCT would then be applied to reconstruct the compressed image. It's like taking a highly compressed version of the sunset image, applying the inverse transform, and getting back the detailed and colorful sunset picture we started with.

# Spectrogram
A spectrogram is like a musical timeline. It shows how loud different pitches are at every moment in a sound.

Example:
Think of a bird's song. A spectrogram would reveal when the bird sings high or low notes and how loud each part is throughout its melody. It's a way to visualize the musical story of the bird's song over time.

The spectrogram formula breakdown:

\[ S(f, t) = \left|X(f, t)\right|^2 \]

- $ S(f, t) $: Spectrogram, representing the magnitude of the spectral content at a specific frequency ($ f $) and time ($ t $).
- $ X(f, t) $: Fourier transform of the signal $ x(t) $ at frequency $ f $ and time $ t $, a complex number with amplitude and phase information.
- $ \left|X(f, t)\right| $: Magnitude (or modulus) of $ X(f, t) $, providing the amplitude of the signal at frequency $ f $ and time $ t $.
- $ \left|X(f, t)\right|^2 $: Square of the magnitude, yielding the power spectral density, a measure of signal power or energy at a specific frequency and time.

This formula is used to compute spectrograms, which visualize how the frequency content of a signal changes over time.

# Bandwidth:
Definition: Bandwidth measures how spread out the frequencies in a signal are. A broader bandwidth indicates a wider range of frequencies.

Its the maximum frequency (Fmax) minus (Fmin) the lowest frequency

# Centroid:
\[ C = \frac{\sum_{i} f_i \cdot A_i}{\sum_{i} A_i} \]

- $ C $: Centroid (average frequency).
- $ f_i $: Frequency of the $ i $-th component.
- $ A_i $: Amplitude (or magnitude) of the $ i $-th component.

Definition: Centroid represents the center of mass of the frequency distribution. It indicates where most of the energy in the spectrum is concentrated.
Its the average frequency.

# Tonnetz
Explanation:
Tonnetz is like a musical map, visually representing how different pitches or chords relate harmonically. Points close together on the hexagonal lattice indicate a strong harmonic connection.

Example:
Consider the chords A minor and E minor. In Tonnetz, they would be visually close, reflecting their harmonic compatibility. On the other hand, A minor and F# major might be farther apart, indicating a less harmonically related pair. Tonnetz simplifies the analysis of harmonic relationships in music.

# Spectral rolloff
the spectral rolloff is simply the frequency that splits the frequency below a certain percentage of the total spectral energy, usually this is at 85%
 The formula is given by:

\[ R = \frac{\sum_{f} S(f)}{\sum_{f} S(f_{\text{total}})} \cdot 100 \leq p \]

- $ R $: Spectral rolloff frequency.
- $ S(f) $: Spectral energy at frequency $ f $.
- $ S(f_{\text{total}}) $: Total spectral energy.
- $ p $: Percentage of total spectral energy (e.g., 85% or 95%).

# Chroma Features:

Chroma features provide insights into the pitch distribution in music, emphasizing tonal qualities. The chromagram has classes.

The chroma features are computed using the chromagram, where each chroma value (C_i) is calculated as:

\[ C_i = \sum_{j} \text{magnitude}(f_j) \]

- $ C_i $: Chroma value for the $ i $-th pitch class.
- $ \text{magnitude}(f_j) $: Magnitude of the $ j $-th frequency component in the spectrum.
- The summation is done over all frequency components that correspond to the $ i $-th pitch class.

This process is repeated for each of the twelve pitch classes to obtain the complete chroma features.

# Spectral Contrast

Spectral Contrast is a feature in audio signal processing that measures the difference in amplitude between peaks and valleys in a frequency spectrum. It helps capture the perceptual contrast between the spectral peaks and background noise, providing information about the tonal quality of the sound.

There is no common formula for spectral contrast.

# Spectral Flatness

Spectral Flatness, also known as tonality or flatness, is a measure of how tonal or noisy a sound is. A perfectly tonal sound (e.g., a sine wave) has high flatness, while a noisy sound has low flatness.

There is no common formula for spectral flatness.

# Zero-Crossing Rate:

Zero-Crossing Rate (ZCR) measures how often an audio signal changes its sign, reflecting its noisiness or tonality.

Formula: $ ZCR = \frac{\text{Number of zero-crossings}}{\text{Total number of samples} - 1} $

- The numerator is the count of times the signal changes sign (crosses zero).
- The denominator is the total number of samples minus one to account for the fact that zero-crossing rate is defined as the rate per sample.


# Tempogram

Tempogram shows the tempo or rhythmic characterisics of music. It provides a way to analyze the variations in tempo, beat, or rhythmic patterns within an audio signal.


The calculation of a tempogram involves computing the local autocorrelation of the audio signal at different time lags. The formula is somewhat complex and involves signal processing techniques, such as Fourier transforms.

$[ T(\tau) = \sum_{n=0}^{N-1} \text{Spec}(\omega_n) \cdot \text{Spec}^*(\omega_n, \tau) $]

- $[( T(\tau) $): Tempogram at time lag $( \tau $]).
- $[( \text{Spec}(\omega_n) $): Short-time Fourier transform of the audio signal at frequency $( \omega_n $]).
- $[( \text{Spec}^*(\omega_n, \tau) $): Complex conjugate of the short-time Fourier transform at frequency $( \omega_n $) and time lag $( \tau $]).
- The summation is done over the signal's discrete frequencies.

# Polyphonic pitch

Polyphonic pitch refers to the ability to identify and analyze multiple pitches in an signal

\[ P_{\text{polyphonic}}(t) = \{f_1, f_2, \ldots, f_n\} \]

- \( P_{\text{polyphonic}}(t) \): Set of identified pitches at time \( t \).
- \( f_1, f_2, \ldots, f_n \): Individual pitch frequencies within the polyphonic mixture.


In [90]:
class AudioFeatureExtractor:
    """
    A class for extracting various audio features from an audio signal.

    Attributes:
        None

    Methods:
        mfccs(data, sfreq):
            Extracts Mel-Frequency Cepstral Coefficients (MFCCs) from the audio signal.

        calculate_spectrograms(audio_clips, n_fft=2048, hop_length=512, win_length=None):
            Calculates spectrograms from a list of audio clips.

        calculate_spectral_features(spectrograms):
            Calculates spectral features (bandwidth and centroid) from a list of spectrograms.

        calculate_spectral_contrast(data, sr, n_fft=2048, hop_length=512):
            Calculates spectral contrast from the audio signal.

        calculate_tonnetz(data, sr):
            Calculates tonnetz features from the audio signal.

        calculate_spectral_rolloff(data, sr, roll_percent=0.85, n_fft=2048, hop_length=512):
            Calculates spectral rolloff from the audio signal.

        calculate_chroma_features(data, sr, n_fft=2048, hop_length=512):
            Calculates chroma features from the audio signal.

        calculate_mel_spectral_contrast(data, sr, n_fft=2048, hop_length=512):
            Calculates mel spectrogram and its contrast from the audio signal.

        calculate_spectral_flatness(data, sr, n_fft=2048, hop_length=512):
            Calculates spectral flatness from the audio signal.

        zero_crossing_rate_features(data, n_fft=2048, hop_length=512):
            Calculates zero-crossing rate from the audio signal.

        rms_energy_features(data):
            Calculates root mean square (RMS) energy from the audio signal.
    """

    def __init__(self):
        pass

    def mfccs(self, data, sr):
        """
        Extracts Mel-Frequency Cepstral Coefficients (MFCCs) from the audio signal.

        Parameters:
            data (numpy.ndarray): The audio signal.
            sr (int): The sampling rate of the audio signal.

        Returns:
            dict: A dictionary containing the mean of each MFCC coefficient.
        """
        mfcc_result = mfcc(y=data, sr=sr)
        datadict = {}
        for var in range(len(mfcc_result)):
            datadict[f'mfcc{var + 1}_mean'] = np.mean(mfcc_result[var, :])
        return datadict

    def calculate_spectrograms(self, audio_clips, n_fft=2048, hop_length=512, win_length=None):
        """
        Calculates spectrograms from a list of audio clips.

        Parameters:
            audio_clips (list): List of audio clips.
            n_fft (int): The number of points for each FFT.
            hop_length (int): The number of samples between successive frames.
            win_length (int): The length of the windowing segments.

        Returns:
            tuple: Two lists - spectrograms and their corresponding dB-scaled versions.
        """
        spectrograms = []
        spectrograms_db = []

        for clip in audio_clips:
            stft_matrix = lr.stft(y=clip, n_fft=n_fft, hop_length=hop_length, win_length=win_length)
            spectrogram = np.abs(stft_matrix)
            spec_db = amplitude_to_db(S=spectrogram, ref=np.max)
            spectrograms.append(spectrogram)
            spectrograms_db.append(spec_db)
        return spectrograms, spectrograms_db

    def calculate_spectral_features(self, spectrograms):
        """
        Calculates spectral features (bandwidth and centroid) from a list of spectrograms.

        Parameters:
            spectrograms (list): List of spectrograms.

        Returns:
            tuple: Two lists - spectral bandwidths and centroids.
        """
        bandwidths = []
        centroids = []

        for spectrogram in spectrograms:
            spec_bw = lr.feature.spectral_bandwidth(S=spectrogram)
            spec_cn = lr.feature.spectral_centroid(S=spectrogram)
            bandwidths.append(spec_bw)
            centroids.append(spec_cn)
        return bandwidths, centroids

    def calculate_spectral_contrast(self, data, sr, n_fft=2048, hop_length=512):
        """
        Calculates spectral contrast from the audio signal.

        Parameters:
            data (numpy.ndarray): The audio signal.
            sr (int): The sampling rate of the audio signal.
            n_fft (int): The number of points for each FFT.
            hop_length (int): The number of samples between successive frames.

        Returns:
            numpy.ndarray: Spectral contrast features.
        """
        datadict = {}
        spectral_contrast = lr.feature.spectral_contrast(y=data, sr=sr, n_fft=n_fft, hop_length=hop_length)
        for var in range(len(spectral_contrast)):
            datadict[f'spectral_contrast{var + 1}_mean'] = np.mean(spectral_contrast[var, :])
        return datadict

    def calculate_tonnetz(self, data, sr):
        """
        Calculates tonnetz features from the audio signal.

        Parameters:
            data (numpy.ndarray): The audio signal.
            sr (int): The sampling rate of the audio signal.

        Returns:
            numpy.ndarray: Tonnetz features.
        """
        tonnetz = lr.feature.tonnetz(y=data, sr=sr)
        datadict = {}
        for var in range(len(tonnetz)):
            datadict[f'tonnetz{var + 1}_mean'] = np.mean(tonnetz[var, :])
        return datadict

    def calculate_spectral_rolloff(self, data, sr, roll_percent=0.85, n_fft=2048, hop_length=512):
        """
        Calculates spectral rolloff from the audio signal.

        Parameters:
            data (numpy.ndarray): The audio signal.
            sr (int): The sampling rate of the audio signal.
            roll_percent (float): Percentage for determining rolloff.
            n_fft (int): The number of points for each FFT.
            hop_length (int): The number of samples between successive frames.

        Returns:
            numpy.ndarray: Spectral rolloff features.
        """
        spectral_rolloff = lr.feature.spectral_rolloff(y=data, sr=sr, roll_percent=roll_percent, n_fft=n_fft, hop_length=hop_length)
        return spectral_rolloff

    def calculate_chroma_features(self, data, sr, n_fft=2048, hop_length=512):
        """
        Calculates chroma features from the audio signal.

        Parameters:
            data (numpy.ndarray): The audio signal.
            sr (int): The sampling rate of the audio signal.
            n_fft (int): The number of points for each FFT.
            hop_length (int): The number of samples between successive frames.

        Returns:
            numpy.ndarray: Chroma features.
        """
        chromagram = chroma_stft(y=data, sr=sr, n_fft=n_fft, hop_length=hop_length)
        datadict = {}
        for var in range(len(chromagram)):
            datadict[f'chromagram{var + 1}_mean'] = np.mean(chromagram[var, :])
        return datadict

    def calculate_mel_spectral_contrast(self, data, sr, n_fft=2048, hop_length=512):
        """
        Calculates mel spectrogram and its contrast from the audio signal.

        Parameters:
            data (numpy.ndarray): The audio signal.
            sr (int): The sampling rate of the audio signal.
            n_fft (int): The number of points for each FFT.
            hop_length (int): The number of samples between successive frames.

        Returns:
            numpy.ndarray: Mel spectrogram contrast features.
        """
        mel_spectrogram = melspectrogram(y=data, sr=sr, n_fft=n_fft, hop_length=hop_length)
        mel_spectral_contrast = lr.feature.spectral_contrast(S=mel_spectrogram)
        datadict = {}
        for var in range(len(mel_spectral_contrast)):
            datadict[f'mel_spectral_contrast{var + 1}_mean'] = np.mean(mel_spectral_contrast[var, :])
        return datadict

    def calculate_spectral_flatness(self, data, sr, n_fft=2048, hop_length=512):
        """
        Calculates spectral flatness from the audio signal.

        Parameters:
            data (numpy.ndarray): The audio signal.
            sr (int): The sampling rate of the audio signal.
            n_fft (int): The number of points for each FFT.
            hop_length (int): The number of samples between successive frames.

        Returns:
            numpy.ndarray: Spectral flatness features.
        """
        flatness = lr.feature.spectral_flatness(y=data, n_fft=n_fft, hop_length=hop_length)
        return flatness

    def zero_crossing_rate_features(self, data, n_fft=2048, hop_length=512):
        """
        Calculates zero-crossing rate from the audio signal.

        Parameters:
            data (numpy.ndarray): The audio signal.
            n_fft (int): The number of points for each FFT.
            hop_length (int): The number of samples between successive frames.

        Returns:
            numpy.ndarray: Zero-crossing rate features.
        """
        zcr = zero_crossing_rate(y=data, frame_length=n_fft, hop_length=hop_length)
        return zcr

    def rms_energy_features(self, data):
        """
        Calculates root mean square (RMS) energy from the audio signal.

        Parameters:
            data (numpy.ndarray): The audio signal.

        Returns:
            numpy.ndarray: RMS energy features.
        """
        rms_energy = lr.feature.rms(y=data)[0]
        return rms_energy

In [91]:
audio_extractor = AudioFeatureExtractor()

In [99]:
#Les notebook code
csv_path = "labels_new.csv"
audio_directory_labeled = "labeled"
audio_directory_unlabeled = "unlabeled"

# Read the CSV file into a DataFrame
df = pd.read_csv(csv_path)

import os
import glob
import pandas as pd
import numpy as np
import librosa as lr

def process_audio_features(audio_directory, labeled=True, df=df):
    # Audio features storage
    audio_features_list = []

    # Set the target length for audio truncation or padding
    lengte = 30 * 22050

    audio_arrays = np.empty((len(os.listdir(audio_directory)), lengte))
    # Process each audio file
    for index, file in enumerate(os.listdir(audio_directory)):
        sr = lr.get_samplerate(f'{audio_directory}/{file}')
        if file.endswith(".wav"):
            # sr = lr.get_samplerate(file)
            file_path = os.path.join(audio_directory, file)
            data, sfreq = lr.load(file_path, sr=None)
            # Truncate or pad the audio
            if len(data) > lengte:
                # Truncate the data
                data = data[:lengte]
            elif len(data) < lengte:
                # Pad with zeros
                padding = lengte - len(data)
                data = np.pad(data, (0, padding), mode='constant')
            
            audio_arrays[index] = data
            
            # Extract MFCC features
            mfcc_features = audio_extractor.mfccs(data, sfreq)
            spectral_contrast_features = audio_extractor.calculate_spectral_contrast(data, sr)
            tonnetz_features = audio_extractor.calculate_tonnetz(data, sr)
            spectral_rolloff_feature = audio_extractor.calculate_spectral_rolloff(data, sr)
            chroma_features = audio_extractor.calculate_chroma_features(data, sr)
            mel_spectral_contrast_features = audio_extractor.calculate_mel_spectral_contrast(data, sr)
            spectral_flatness_feature = audio_extractor.calculate_spectral_flatness(data, sr)
            zero_crossing_feature = audio_extractor.zero_crossing_rate_features(data)
            rms_energy = audio_extractor.rms_energy_features(data)
            
       # Combine all features into a dictionary
            features_dict = {
                'filename': file, 
                **mfcc_features,
                'rms_energy': np.mean(rms_energy),
                **spectral_contrast_features,
                **tonnetz_features,
                'spectral_rolloff': np.mean(spectral_rolloff_feature[0]),
                **chroma_features,
                **mel_spectral_contrast_features,
                'spectral_flatness': np.mean(spectral_flatness_feature[0]),
                'zcr': np.mean(zero_crossing_feature[0])
            }
                # Append the features dictionary to the list
            audio_features_list.append(features_dict) 
            
    spectograms, spectogram_features = audio_extractor.calculate_spectrograms(audio_arrays)
    bandwidths, centroids = audio_extractor.calculate_spectral_features(spectograms)

    bandwiths = pd.DataFrame({'bandwith': [np.mean(bandwidth[0]) for bandwidth in bandwidths]})
    centroids = pd.DataFrame({'centroid': [np.mean(centroid[0]) for centroid in centroids]})
    # Convert the list of feature dictionaries to a DataFrame
    audio_features_df = pd.DataFrame(audio_features_list)
    audio_features_df = pd.concat([audio_features_df, bandwiths], axis=1)
    audio_features_df = pd.concat([audio_features_df, centroids], axis=1)
    # Merge the original DataFrame with the new features DataFrame based on the 'filename' column
    if labeled:
        audio_features_df = pd.merge(df, audio_features_df, on='filename', how='left')


    return audio_features_df

# Process audio features
labeled_df = process_audio_features(audio_directory_labeled)
display(labeled_df)

unlabeled_df = process_audio_features(audio_directory_unlabeled, labeled=False)
display(unlabeled_df)

,filename,genre,mfcc1_mean,mfcc2_mean,mfcc3_mean,mfcc4_mean,mfcc5_mean,mfcc6_mean,mfcc7_mean,mfcc8_mean,...,mel_spectral_contrast2_mean,mel_spectral_contrast3_mean,mel_spectral_contrast4_mean,mel_spectral_contrast5_mean,mel_spectral_contrast6_mean,mel_spectral_contrast7_mean,spectral_flatness,zcr,bandwith,centroid
0,m00248.wav,metal,-75.517517,81.911423,-22.081079,69.876999,-11.740438,25.740246,-18.518965,27.027712,...,2.714730,9.285961,13.318006,15.142012,17.020385,57.419416,0.018547,0.158776,2337.227557,2656.165734
1,m00230.wav,country,-89.736382,36.286205,11.469535,37.495586,9.199137,3.740519,2.840358,6.811540,...,4.663522,12.311560,17.618104,28.147029,26.801658,22.182678,0.093471,0.212375,3053.066772,3760.424488
2,m00637.wav,hiphop,-122.780525,95.061287,-29.363251,46.780045,-15.998561,27.117586,-13.113777,20.258003,...,6.453961,10.992537,16.350120,21.788517,20.466351,50.293757,0.013041,0.113229,2181.589027,2290.107318
3,m00627.wav,metal,-57.683388,101.432320,-41.485245,55.130600,-23.349279,28.151098,-12.139105,18.150204,...,4.042038,9.291089,13.214748,17.212025,17.637471,58.275039,0.012321,0.141317,2092.439557,2378.392048
4,m00138.wav,reggae,-198.632797,102.413567,-10.031449,30.802383,-3.310606,20.276924,-5.637373,28.095631,...,8.111595,11.985461,21.690643,26.647311,26.212014,47.129611,0.005168,0.070323,2295.466240,1995.600514
5,m00192.wav,classical,-85.264992,118.423058,-31.472771,20.282015,-4.417709,6.922598,-10.950942,-1.620085,...,5.707315,9.457773,16.339001,23.798978,26.500622,37.537164,0.010484,0.111613,1965.028715,1878.774030
6,m00429.wav,hiphop,-109.509193,97.389565,-20.619621,37.366673,-3.125998,28.611204,-14.169424,23.862112,...,4.376860,8.659314,15.042835,21.151530,22.051505,50.071179,0.011507,0.104502,2327.237804,2272.834439
7,m00623.wav,reggae,-94.140526,68.229942,3.975481,7.631387,12.791034,-3.185735,-3.907444,3.817600,...,7.047003,16.691960,18.115008,23.019129,24.171044,21.322361,0.065537,0.136075,2974.065401,3086.186001
8,m00002.wav,jazz,-298.807953,112.078224,6.485770,28.386517,-6.764679,16.651894,-11.809684,12.746783,...,6.739004,17.720329,31.856210,37.582171,36.234603,42.188263,0.001252,0.051222,1919.917650,1451.498371
9,m00039.wav,reggae,-169.243668,110.447716,-8.553957,43.898697,0.266454,26.646509,-14.365674,13.026837,...,8.001072,15.869485,22.225069,29.004593,27.931632,43.075836,0.004306,0.072178,2019.252686,1811.358216


,filename,mfcc1_mean,mfcc2_mean,mfcc3_mean,mfcc4_mean,mfcc5_mean,mfcc6_mean,mfcc7_mean,mfcc8_mean,mfcc9_mean,...,mel_spectral_contrast2_mean,mel_spectral_contrast3_mean,mel_spectral_contrast4_mean,mel_spectral_contrast5_mean,mel_spectral_contrast6_mean,mel_spectral_contrast7_mean,spectral_flatness,zcr,bandwith,centroid
0,m00003.wav,-82.501259,97.344116,-34.373585,71.405922,-3.283182,17.367373,-9.023832,16.011181,-19.003857,...,5.011253,13.892055,18.007974,21.183092,21.028454,58.034254,0.009073,0.120159,2070.908728,2254.451748
1,m00012.wav,-1.925410,72.695557,-32.789642,63.592033,-18.557953,25.872955,-13.854105,18.064384,-11.945406,...,6.322091,13.544996,16.113373,22.568365,18.241123,55.859045,0.026390,0.178119,2286.246872,2908.260266
2,m00013.wav,-287.520996,101.737930,-35.368999,41.282764,-12.745122,17.284992,-13.978298,16.311884,-5.599028,...,4.346955,16.482222,24.210026,32.560252,34.102601,59.447518,0.002209,0.123492,1865.851544,1953.012399
3,m00043.wav,-120.127808,91.287666,-38.794960,75.003784,-3.806012,22.666491,-8.123724,23.654573,-14.638494,...,4.574703,13.135090,15.795038,18.074718,19.134837,60.384607,0.008840,0.126103,2073.302486,2384.812610
4,m00044.wav,-437.525238,170.971405,7.386171,-3.108705,0.777301,-11.135056,-12.523806,-6.900928,-5.850053,...,4.602050,18.591778,23.050981,33.758989,36.850383,40.763202,0.000153,0.052112,899.979779,790.478225
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100,m00971.wav,-246.218369,149.443878,-41.973816,11.606412,-10.287585,-4.018404,-7.138946,-10.137027,-9.425965,...,6.321750,12.820653,17.553917,24.904399,30.941367,37.337512,0.006206,0.091588,1588.161024,1525.601266
101,m00973.wav,-8.531341,54.251755,20.963310,13.099079,9.721153,6.015052,10.720785,6.102119,0.694264,...,7.653338,10.039850,19.457696,25.698457,25.316437,18.259208,0.088824,0.161897,3295.553789,3671.260489
102,m00988.wav,-56.091187,36.468166,11.945320,-6.441751,3.778163,5.284157,3.394784,4.265098,7.993917,...,6.389734,16.538572,24.927608,25.531409,23.668680,16.240189,0.089230,0.167399,3361.504206,3940.349921
103,m00991.wav,-406.965424,163.139465,-2.413815,32.456657,7.907838,9.677142,-5.489042,9.523497,-3.102821,...,8.248197,19.474204,27.322492,31.308818,33.020523,45.577339,0.000300,0.044227,1404.132189,983.761841


In [ ]:
import os
import pandas as pd
import numpy as np
import librosa as lr

# Define the path to the audio directory
audio_directory = "unlabeled"

# Create an empty DataFrame to store the audio features
df = pd.DataFrame()

# Set the target length for audio truncation or padding
target_length = 30 * 22050

# Process each audio file
for file in os.listdir(audio_directory):
    if file.endswith(".wav"):
        file_path = os.path.join(audio_directory, file)
        data, sfreq = lr.load(file_path, sr=None)

        # Truncate or pad the audio
        if len(data) > target_length:
            # Truncate the data
            data = data[:target_length]
        elif len(data) < target_length:
            # Pad with zeros
            padding = target_length - len(data)
            data = np.pad(data, (0, padding), mode='constant')

        # Extract MFCC features
        mfcc_features = audio_extractor.mfccs(data, sfreq)

        # moet meer bij maar duurt anders te lang
        rms_energy = audio_extractor.rms_energy_features(data)
        # Combine all features into a dictionary
        features_dict = {
            'filename': file, 
            **mfcc_features,
            'rms_energy': np.mean(rms_energy)
        }

        df = pd.concat([df, pd.DataFrame([features_dict])], ignore_index=True)


df_sorted = df.sort_values(by='filename')

display(df_sorted.head())





In [ ]:

def scaler(df):
    df.set_index('filename', inplace=True)
    """
    Standardizes numeric columns of a DataFrame based on mean and standard deviation.

    Parameters:
    - df (pd.DataFrame): Input DataFrame containing numeric columns to be standardized.

    Returns:
    - pd.DataFrame: A new DataFrame with standardized numeric columns.
    """
    # Calculate mean and standard deviation
    mean_vals = df.mean(numeric_only=True)
    std_vals = df.std(ddof=0, numeric_only=True)

    # Standardize the data
    scaled_df = (df - mean_vals) / std_vals

    scaled_df['filename'] = df.index


    return scaled_df


In [ ]:
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
from IPython.display import display  

df = scaler(df_sorted)
display(df.head())


In [ ]:
df.drop("filename", axis=1, inplace=True)
df

In [ ]:
# overgenomen uit ml les
import matplotlib.pyplot as plt
import numpy as np
from sklearn.cluster import KMeans, AgglomerativeClustering, DBSCAN, Birch, MeanShift
inertia = []
k = range(1, 11)

for i in k:
    model = KMeans(n_clusters=i, n_init=10)

    model.fit(df)

    inertia.append(model.inertia_)

plt.plot(k, inertia, '-x')
plt.xlabel('Aantal (k) clusters')
plt.ylabel('inertia')
plt.xticks(k)
plt.show()

hier kunnen we zien dat het elleboog op het punt k = 3 staat. dit betekent dat de beste aantal clusters 3 is.

In [ ]:
data = df
data

# Fantasy Cup Spelers Dataset

| Player    | Speed    | Defence     | Pas    | Shot     | Creativeness |
|-----------|----------|-------------|--------|----------|--------------|
| Vikram    | 90       | 70          | 80     | 75       | 85           |
| Bob       | 60       | 90          | 50     | 40       | 50           |
| Mark      | 80       | 65          | 90     | 70       | 80           |
| Sami      | 95       | 40          | 60     | 85       | 70           |
| Daan      | -        | 85          | -      | -        | -            |
| Penaldo   | 75       | 30          | 60     | 95       | 40           |
| Pessi     | 70       | 50          | 85     | 60       | 90           |




# Initialization:

Choose the number of clusters (K) that you want to create in your dataset.
Randomly initialize K cluster centroids. These centroids represent the initial centers of the clusters.
Assignment Step:

For each data point in the dataset, calculate the distance to each centroid.
Assign the data point to the cluster whose centroid is the closest using Euclidean distance,

Recalculate the centroids of each cluster by taking the mean of all data points assigned to that cluster.

Repeat the assignment and update steps until convergence. Convergence occurs when the centroids no longer change significantly or when a certain number of iterations are reached.
Output:

The algorithm outputs K clusters, each represented by its centroid.

note* even though Daan has NaN values he wil stil be asigned to a centroid based on existing features




In [ ]:
from sklearn.cluster import KMeans, AgglomerativeClustering, DBSCAN, Birch, MeanShift
from sklearn.preprocessing import StandardScaler

from sklearn.cluster import KMeans, AgglomerativeClustering, DBSCAN, Birch, MeanShift
from sklearn.preprocessing import StandardScaler
import numpy as np

class Clustering:
    def __init__(self, data):
        """
        Initializes the Clustering object with input data.

        Parameters:
        - data (numpy.ndarray): Input data for clustering.
        """
        self.data = data
        self.cluster_labels = None



        def calculate_silhouette_score(self):
            if self.cluster_labels is not None:
                if len(np.unique(self.cluster_labels)) > 1:
                    self.silhouette_score = silhouette_score(self.data, self.cluster_labels)
                else:
                    print("Silhouette score cannot be calculated with only one cluster.")
            else:
                print("Cluster labels not available. Run a clustering algorithm first.")

    def cluster_kmeans(self, n_clusters=3):
        """
        Performs KMeans clustering on the input data.

        Parameters:
        - n_clusters (int): Number of clusters.

        Returns:
        - numpy.ndarray: An array containing the input data and assigned cluster labels.
        """
        kmeans = KMeans(n_clusters=n_clusters, random_state=42)
        self.cluster_labels = kmeans.fit_predict(self.data)
        return np.column_stack((self.data, self.cluster_labels))

    def cluster_agglomerative(self, n_clusters=3):
        """
        Performs Agglomerative clustering on the input data.

        Parameters:
        - n_clusters (int): Number of clusters.

        Returns:
        - numpy.ndarray: An array containing the input data and assigned cluster labels.
        """
        agglomerative = AgglomerativeClustering(n_clusters=n_clusters)
        self.cluster_labels = agglomerative.fit_predict(self.data)
        return np.column_stack((self.data, self.cluster_labels))

    def cluster_dbscan(self, eps=0.5, min_samples=5):
        """
        Performs DBSCAN clustering on the input data.

        Parameters:
        - eps (float): The maximum distance between two samples for one to be considered as in the neighborhood of the other.
        - min_samples (int): The number of samples (or total weight) in a neighborhood for a point to be considered as a core point.

        Returns:
        - numpy.ndarray: An array containing the input data and assigned cluster labels.
        """
        dbscan = DBSCAN(eps=eps, min_samples=min_samples)
        self.cluster_labels = dbscan.fit_predict(self.data)
        return np.column_stack((self.data, self.cluster_labels))

    def cluster_birch(self, n_clusters=3):
        """
        Performs Birch clustering on the input data.

        Parameters:
        - n_clusters (int): Number of clusters.

        Returns:
        - numpy.ndarray: An array containing the input data and assigned cluster labels.
        """
        birch = Birch(n_clusters=n_clusters)
        self.cluster_labels = birch.fit_predict(self.data)
        return np.column_stack((self.data, self.cluster_labels))

    def cluster_meanshift(self, bandwidth=0.5):
        """
        Performs MeanShift clustering on the input data.

        Parameters:
        - bandwidth (float): Bandwidth parameter for MeanShift.

        Returns:
        - numpy.ndarray: An array containing the input data and assigned cluster labels.
        """
        meanshift = MeanShift(bandwidth=bandwidth)
        self.cluster_labels = meanshift.fit_predict(self.data)
        return np.column_stack((self.data, self.cluster_labels))


In [ ]:
unlabeled_data = data
unlabeled_data

In [ ]:
from sklearn.cluster import KMeans


k = 3  

kmeans = KMeans(n_clusters=k, n_init=10, random_state=42)

unlabeled_data['cluster'] = kmeans.fit_predict(unlabeled_data)

print(unlabeled_data['cluster'].value_counts())

hier voeren we Kmeans op de unlabeled dataset en voegen de clusters toe als kolommen.

In [ ]:
unlabeled_cluster = unlabeled_data.groupby('cluster').agg('mean')

display(unlabeled_cluster)


In [ ]:
labeled = df1
print(labeled)

In [ ]:
label_scaled = scaler(labeled)

label_scaled['genre'] = labeled['genre']

label_scaled = label_scaled.groupby('genre').agg('mean', numeric_only=True)
# Drop 'index' en 'sfreq' kolommen

display(label_scaled)


In [ ]:
from scipy.spatial.distance import euclidean

def bereken_afstanden(genre_data, cluster_data):
    afstanden = []

    for genre_row in genre_data.iterrows():
        genre_name, genre_values = genre_row
        for cluster_row in cluster_data.iterrows():
            cluster_name, cluster_values = cluster_row
            distance = euclidean(genre_values, cluster_values)
            afstanden.append((genre_name, cluster_name, distance))

    return afstanden

afstanden = bereken_afstanden(label_scaled, unlabeled_cluster)

for genre_name, cluster_name, distance in afstanden:
    print(f"Afstand tussen {genre_name} en {cluster_name}: {distance}")

In [ ]:
from scipy.spatial.distance import cityblock

def bereken_manhattan_afstanden(genre_data, cluster_data):
    afstanden = []

    for genre_row in genre_data.iterrows():
        genre_name, genre_values = genre_row
        for cluster_row in cluster_data.iterrows():
            cluster_name, cluster_values = cluster_row
            distance = cityblock(genre_values, cluster_values)
            afstanden.append((genre_name, cluster_name, distance))

    return afstanden

manhattan_afstanden = bereken_manhattan_afstanden(label_scaled, unlabeled_cluster)

for genre_name, cluster_name, distance in manhattan_afstanden:
    print(f"Manhattan-afstand tussen {genre_name} en {cluster_name}: {distance}")

In [ ]:
def bepaal_genres_voor_clusters(genre_data, cluster_data, afstanden_functie):
    toewijzingen = []

    for cluster_row in cluster_data.iterrows():
        cluster_name, cluster_values = cluster_row
        min_afstand = float('inf') 
        op_een_na_min_afstand = float('inf')  
        toegewezen_genre = None
        op_een_na_toegewezen_genre = None

        for genre_row in genre_data.iterrows():
            genre_name, genre_values = genre_row
            afstand = afstanden_functie(genre_values, cluster_values)

            if afstand < min_afstand:
                op_een_na_min_afstand = min_afstand
                op_een_na_toegewezen_genre = toegewezen_genre

                min_afstand = afstand
                toegewezen_genre = genre_name
            elif afstand < op_een_na_min_afstand:
                op_een_na_min_afstand = afstand
                op_een_na_toegewezen_genre = genre_name

        toewijzingen.append((cluster_name, toegewezen_genre, min_afstand, op_een_na_toegewezen_genre, op_een_na_min_afstand))

    return toewijzingen

toewijzingen_euclidisch = bepaal_genres_voor_clusters(label_scaled, unlabeled_cluster, euclidean)

toewijzingen_manhattan = bepaal_genres_voor_clusters(label_scaled, unlabeled_cluster, cityblock)

print("Toewijzingen op basis van Euclidische afstanden:")
for cluster_name, genre_name, afstand, op_een_na_genre, op_een_na_afstand in toewijzingen_euclidisch:
    print(f"Cluster {cluster_name} is toegewezen aan genre {genre_name} met een afstand {afstand}. Op één na dichtstbijzijnde genre: {op_een_na_genre} met een afstand {op_een_na_afstand}")

print("\nToewijzingen op basis van Manhattan-afstanden:")
for cluster_name, genre_name, afstand, op_een_na_genre, op_een_na_afstand in toewijzingen_manhattan:
    print(f"Cluster {cluster_name} is toegewezen aan genre {genre_name} met een afstand {afstand}. Op één na dichtstbijzijnde genre: {op_een_na_genre} met een afstand {op_een_na_afstand}")

In [ ]:
def plot_vis(data1, data2, x, y):


    plt.figure(figsize=(15, 6))

    sns.scatterplot(data=data1,
                    x=x,
                    y=y,
                    hue='cluster',
                    palette='viridis',
                    marker='o',  
                    legend='full')
    
    sns.scatterplot(data=data2,
                    x=x,
                    y=y,
                    hue='genre',
                    marker='D',  
                    legend='full')
    
    plt.title(f'Scatter Plot van {x} en {y} met Cluster- en Genrevergelijking')
    plt.xlabel(x)
    plt.ylabel(y)
    plt.legend(loc='upper right')
    plt.grid(True)
    plt.show()


In [ ]:
unlabeled_cluster.reset_index(inplace=True)
display(unlabeled_cluster.head())
label_scaled.reset_index(inplace=True)
display(label_scaled.head())

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

clusters = [0]

genres = ['disco', 'pop']


x = 'rms_energy'
y = 'mfcc11_mean'

plot_vis(
    data1=unlabeled_data[unlabeled_data['cluster'].isin(clusters)],
    data2=label_scaled[label_scaled['genre'].isin(genres)],
    x=x,
    y=y
)


In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

clusters = [1]

genres = ['metal', 'hiphop']

plot_vis(
    data1=unlabeled_data[unlabeled_data['cluster'].isin(clusters)],
    data2=label_scaled[label_scaled['genre'].isin(genres)],
    x=x,
    y=y
)

In [ ]:
clusters = [2]

genres = ['jazz', 'classical']

x = 'rms_energy'
y = 'mfcc11_mean'

plot_vis(
    data1=unlabeled_data[unlabeled_data['cluster'].isin(clusters)],
    data2=label_scaled[label_scaled['genre'].isin(genres)],
    x=x,
    y=y
)

# PCA Voorbeeld

Om een voorbeeld te geven van het uitvoeren van PCA hebben we eerst een dataset nodig. Voor het gemak maken wij gebruik van een kleine zelf verzonnen dataset.
|       |   X1   |   X2   |
|-------|:-----:|:-----:|
|   1   |   4   |   1   |
|   2   |   2   |   3   |
|   3   |   7   |   6   |
|   4   |   5   |   4   |
|   5   |   9   |   8   |


## Stap 1: Standaardiseren

De eerste stap voor het uitvoeren van PCA is het standaardiseren van de bovenstaande dataset. Dit doen wij aan de hand van de formule voor de Z-score. $$
Z = \frac{{X - \mu}}{{\sigma}}
$$

Door het toepassen van de Z-score krijgen wij de volgende gestandaardiseerde dataset:

|   | xz1  | xz2  |
|---|---------|---------|
| 1 | -0.6325 | -1.2649 |
| 2 | -1.2649 | 0       |
| 3 | 0.6325  | 0.6325  |
| 4 | 0       | -0.6325 |
| 5 | 1.8974  | 1.2649  |


## Stap 2: Covariantiematrix
Vervolgen bereken wij de covariantie matrix van de gestandaardiseerde dataset. Dit doen wij aan de hand van de volgende formule:

$$
Cov(X_i, X_j) = \frac{\sum_{k=1}^{n}(X_i^k - \bar{X_i})(X_j^k - \bar{X_j})}{n-1}
$$

Wanneer wij de bovenstaande formule toepassen krijgen wij het volgende matrix:

\begin{bmatrix} 6.75 & 0.5 \\ 0.5 & 5.0 \end{bmatrix}






## Stap 3: Eigenwaarden en Eigenvectoren
In stap 3 berekenen wij de eigenwaarden en eigenvectoren.

Eigenwaarden:
\begin{align*}
\lambda_1 &= 1.9 \\
\lambda_2 &= 0.8
\end{align*}



Eigenvectoren:
\begin{align*}
v_1 &= \begin{bmatrix} 0.84 \\ 
0.54 \end{bmatrix} \quad (\text{bij } \lambda_1) \\
v_2 &= \begin{bmatrix} -0.54 \\
0.84 \end{bmatrix} \quad (\text{bij } \lambda_2)
\end{align*}


## Stap 4: Feature vector
Wanneer we kijken naar stap 3 zien we dat vector 1 de meeste significantie heeft, dit is dus de vector met de hoogste eigenwaarde. We selecteren daarom v1 als ons eerst principal component. 
v_1 &= \begin{bmatrix} 0.84 \\ 
0.54 \end{bmatrix} 


## Stap 5: Projectie
Voor stap 5 gaan we de gestandaardiseerde gegevens vermenigvuldigen met de gekozen feature vector. Vervolgens krijgen de nieuwe dataset:
$$[-0.6325, -1.2649, -1.2649, 0.6325, 0.6325] \cdot [0.84, 0.54] = [-1.6326, -1.2649, -0.0810, 2.6305] $$
Zo komen wij tot een nieuwe dataset die is gereduceerd tot één dimensie, de eerste principale component.

|                       | Nieuwe Dataset |
|-----------------------|-----------------|
|           1           |     -1.6326     |
|           2           |     -1.2649     |
|           3           |     -0.0810     |
|           4           |      2.6305     |





Voor het uitvoeren van een simpele PCA hebben wij de stappen gevolgd die beschreven staan in het volgende artikel. (Jaadi, 2023) 

Nu dat wij een voorbeeld hebben gegeven van een simpele principal componenent analyse gaan wij dit nu uitvoeren op onze eigen dataset.


In [ ]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

# Stap 1: Standaardiseren
#De data is eerder al gestandaardiseerd. Deze data kunnen we hier hergebruiken
scaled_df = data

In [ ]:
# Stap 2: Covariantiematrix
# We maken een covariantiematrix aan de hand van no.cov
cov_matrix = np.cov(scaled_df, rowvar=False)

In [ ]:
# Stap 3: Eigenwaarden en Eigenvectoren
eigenvalues, eigenvectors = np.linalg.eig(cov_matrix)

# Nu sorteren wij de waardes
sorted_indices = np.argsort(eigenvalues)[::-1]
eigenvalues = eigenvalues[sorted_indices]
eigenvectors = eigenvectors[:, sorted_indices]


In [ ]:
# Aantal principal compponents kiezen
plt.plot(eigenvalues, marker='o')
plt.xlabel('Index')
plt.ylabel('Eigenvalue')
plt.title('Eigenvalues voor het bepalen van het aantal principal components')
plt.show()


In [ ]:
# Stap 4: Feature vector
pca_num = 3
principal_components = eigenvectors[:, -pca_num:]




In [ ]:
# Stap 5: Projectie
pca = PCA(n_components=pca_num)
data_pca = pca.fit_transform(scaled_df)


In [ ]:
# Resultaten toevoegen aan dataframe
unlabeld_new = unlabeled_data.copy()
for i in range(pca_num):
    unlabeld_new[f'PC{i+1}'] = data_pca[:, i]

In [ ]:
# Toon het resultaat
display(unlabeled_data.head())


In [ ]:
#PCA toevoegen aan dataset
df_pca = scaled_df.copy()
for i in range(pca_num):
    df_pca[f'PC{i+1}'] = data_pca[:, i]

In [ ]:
pca_df = df_pca[['PC1', 'PC2', 'PC3']]


In [ ]:
#Getransformeerde dataset
display(pca_df.head())

Bronnen:
Bronnen lijst PCA Daan
Wikipedia-bijdragers. (2023, 31 juli). Hoofdcomponentenanalyse. Wikipedia. Geraadpleegd op 21-1-2024 van https://nl.wikipedia.org/wiki/Hoofdcomponentenanalyse 
Jaadi, Z. (2023, 29 maart). A Step-by-Step Explanation of Principal Component Analysis (PCA). Built In. Geraadpleegd op 21-1-2024 van https://builtin.com/data-science/step-step-explanation-principal-component-analysis
Beat and Tempo — Librosa 0.10.1 Documentation. (z.d.). https://librosa.org/doc/latest/beat.html
Filters — Librosa 0.10.1 documentation. (z.d.). https://librosa.org/doc/latest/filters.html#filter-bank-construction
Librosa.feature.spectral_centroid — Librosa 0.10.1 documentation. (z.d.). https://librosa.org/doc/latest/generated/librosa.feature.spectral_centroid.html
Librosa.feature.spectral_bandwidth — Librosa 0.10.1 Documentation. (z.d.). https://librosa.org/doc/latest/generated/librosa.feature.spectral_bandwidth.html
Librosa.feature.spectral_flatness — Librosa 0.10.1 Documentation. (z.d.). https://librosa.org/doc/latest/generated/librosa.feature.spectral_flatness.html
Librosa.feature.spectral_rolloff — Librosa 0.10.1 Documentation. (z.d.). https://librosa.org/doc/latest/generated/librosa.feature.spectral_rolloff.html
Librosa.feature.tonnetz — Librosa 0.10.1 Documentation. (z.d.). https://librosa.org/doc/latest/generated/librosa.feature.tonnetz.html
Librosa.feature.zero_crossing_rate — Librosa 0.10.1 Documentation. (z.d.). https://librosa.org/doc/latest/generated/librosa.feature.zero_crossing_rate.html
Librosa.feature.Tempogram — Librosa 0.10.1 documentation. (z.d.). https://librosa.org/doc/latest/generated/librosa.feature.tempogram.html
Scipy.spatial.distance.cityblock — SciPY v1.12.0 Manual. (z.d.). https://docs.scipy.org/doc/scipy/reference/generated/scipy.spatial.distance.cityblock.html
Scipy.spatial.distance.euclidean — SciPY v1.12.0 manual. (z.d.). https://docs.scipy.org/doc/scipy/reference/generated/scipy.spatial.distance.euclidean.html
Scipy.spatial.distance.euclidean — SciPY v1.12.0 manual. (z.d.-b). https://docs.scipy.org/doc/scipy/reference/generated/scipy.spatial.distance.euclidean.html
Scipy.spatial.distance.cityblock — SciPY v1.12.0 Manual. (z.d.-b). https://docs.scipy.org/doc/scipy/reference/generated/scipy.spatial.distance.cityblock.html



# Feature run probleem
Extract other features using the AudioFeatureExtractor class
            spectrograms, _ = audio_extractor.calculate_spectrograms([data])
            bandwidths, centroids = audio_extractor.calculate_spectral_features(spectrograms)
            spectral_contrast = audio_extractor.calculate_spectral_contrast(data, sfreq)
            tonnetz = audio_extractor.calculate_tonnetz(data, sfreq)
            spectral_rolloff = audio_extractor.calculate_spectral_rolloff(data, sfreq)
            chroma_features = audio_extractor.calculate_chroma_features(data, sfreq)
            mel_spectral_contrast = audio_extractor.calculate_mel_spectral_contrast(data, sfreq)
            spectral_flatness = audio_extractor.calculate_spectral_flatness(data, sfreq)
            zcr = audio_extractor.zero_crossing_rate_features(data)
            tempogram_result = audio_extractor.calculatetempogram(data, sfreq)
            f0, , _ = audio_extractor.calculate_polyphonic_pitch(data, sfreq)
            rms_energy = audio_extractor.rms_energy_features(data)

            # Combine all features into a dictionary
            features_dict = {
                **mfcc_features,
                'mean_bandwidth': np.mean(bandwidths),
                # ... (add other features to the dictionary)
                'tempogram_result': tempogram_result,
                'polyphonic_pitch': f0,
                'rms_energy': r

                We hebben deze features niet kunnen runnen omdat de notebook na 24 uur laden het nog niet was geladen, hierdoor hebben we sommige delen minder accuraat kunnen doen
                